# Datastory Trachea-, bronchiën- en longkanker



# Inleiding

Kanker is een ingrijpende ziekte die vele mensen wereldwijd raakt. De oorzaak van kanker hangt af van de soort kanker. Voor trachea, bronchus en long (TBL) kanker zijn er  twee hoofdoorzaken: roken en slechte luchtkwaliteit (Safiri et al., 2021). In dit dataverhaal wordt gekeken naar deze twee perspectieven. Wordt TBL-kanker veroorzaakt door de slechte luchtkwaliteit of door roken? 
Om dit te onderzoeken bekijken we de volgende dataset over kankertype en doden: https://www.kaggle.com/datasets/antimoni/cancer-deaths-by-country-and-type-1990-2016 
The Cancer Deaths by Country and Type dataset bevat data van 18 verschillende soorten kanker (prostaatkanker, leverkanker etc.) en het dodental per soort kanker per jaar en per land. De data is van 1990-2016. De data is verzameld van de WHO (World Health Organisation).
Dit gaan we dan vergelijken met de dataset van uitstoot:
https://www.kaggle.com/datasets/thedevastator/global-fossil-co2-emissions-by-country-2002-2022

De Emissions by Country dataset bevat mondiale data van fossiele brandstof CO2-uitstoot per land per jaar. De dataset bevat totale CO2-uitstoot per jaar per land maar ook uitstoot per aandeel gas, olie, kolen of uitstoot per inwoner. De dataset is van 1750-2021.

En de dataset over roken: 
https://www.kaggle.com/datasets/mexwell/us-smoking-trend

De Global Smoking Trend dataset bevat data over het aantal rokers per land per jaar. De dataset is van 1980-2012.


# 

In [374]:
# Import packages
import pandas as pd
import plotly.express as px
import plotly.graph_objs as go 
import matplotlib.pyplot as plt
import seaborn as sns
from plotly.subplots import make_subplots

## Fijnstof

Luchtvervuiling en met name fijnstof is een van de grootste veroorzakers van longkanker.

Luchtvervuiling en met name fijnstof, wat veroorzaakt wordt door onvolledige verbranding van fossiele brandstoffen (Cohen & Pope, 1995), is een grote veroorzaker van onder andere longkanker. In 2016 was met 19 procent van alle kanker-gerelateerde overlijdens TBL-kanker (Trachea, bronchiën en longkanker) de grootste doodsoorzaak onder alle kankersoorten (Safiri et al., 2021). De globale uitstoot stijgt, en met die uitstoot dus ook de uitstoot van fijnstof. Dit leidt tot een toename in TBL kanker, hieronder is een aantal grafieken neergezet die goed laten zien wat de uitstoot van fijnstof voor invloed heeft op TBL-kanker.

In [375]:
# importeer gecleande dataframes 
year_lung_cancer = pd.read_csv("csv/df/cancer_cleaned_data.csv")
year_total_emissions = pd.read_csv("csv/df/emissions_cleaned_data.csv")

# Create a subplot figure with 1 row and 2 columns
fig = make_subplots(rows=1, cols=2, subplot_titles=('Tracheale, bronchus- en longkanker per jaar', 'Totale Emissies Per Jaar Wereldwijd'))

# Add the lung cancer data trace to the first subplot
fig.add_trace(go.Scatter(x=year_lung_cancer['Year'], y=year_lung_cancer["Tracheal, bronchus, and lung cancer "],
                         mode='lines', name='Tracheale, bronchus- en longkanker'),
              row=1, col=1)

# Add the total emissions data trace to the second subplot
fig.add_trace(go.Scatter(x=year_total_emissions['Year'], y=year_total_emissions['Total'],
                         mode='lines', name='Totale Emissies'),
              row=1, col=2)

# Update the layout for the entire figure
fig.update_layout(
    title_text='Vergelijking van Tracheale, bronchus- en longkanker en Totale Emissies per jaar',
    showlegend=False
)

# Update x-axis and y-axis titles for each subplot
fig.update_xaxes(title_text='Jaar', row=1, col=1)
fig.update_yaxes(title_text='Tracheale, bronchus- en longkanker', row=1, col=1)
fig.update_xaxes(title_text='Jaar', row=1, col=2)
fig.update_yaxes(title_text='Totale Emissies', row=1, col=2)

# Show the plot
fig.show()

# Wereld visualisatie

Er zijn ook erg veel culturele verschillen op het gebied van longkanker, op deze wereldkaarten valt te zien welke gebieden in de wereld het meeste te maken hebben met longkanker en luchtvervuiling.

In [376]:
# importeer gecleande datasets

df_lung_per_capita = pd.read_csv("csv/df/lung_per_capita.csv")
df_emissions_per_capita = pd.read_csv("csv/df/emissions_per_capita.csv")

# Create the choropleth map for lung cancer rates
fig_lung = px.choropleth(df_lung_per_capita,
                         locations='Code',
                         color='Lung Cancer Per Capita',
                         hover_name='Country',
                         color_continuous_scale=px.colors.sequential.Plasma,
                         title='Lung Cancer Rates per Capita by Country')

# Update layout for a larger map
fig_lung.update_layout(
    title_text='TBL-Kanker Cijfers per Capita per Land',
    title_x=0.5,
    geo=dict(
        showframe=False,
        showcoastlines=False,
        projection_type='equirectangular'
    ),
    height=600,
    coloraxis_colorbar=dict(
        title="TBL-kanker<br>Per Capita",
    )
)

# Create the choropleth map for emissions
fig_emissions = px.choropleth(df_emissions_per_capita,
                              locations='Code',
                              color='Emissions Per Capita',
                              hover_name='Country',
                              color_continuous_scale=px.colors.sequential.Plasma,
                              title='Emissions per Capita by Country'
                              )

# Update layout for a larger map
fig_emissions.update_layout(
    title_text='Emissies per Capita per Land',
    title_x=0.5,
    geo=dict(
        showframe=False,
        showcoastlines=False,
        projection_type='equirectangular'
    ),
    height=600,
    coloraxis_colorbar=dict(
        title="Emissies<br>Per Capita",
    )
)

# Show the figures
fig_lung.show()
fig_emissions.show()

# Bar Chart verdeling emissies 

Om het nog duidelijker te maken, staat het hieronder in twee bar plots verdeeld. Wat opvalt, is dat de grafiek niet geheel overeenkomt, dit duidt dus wel op andere correlaties die nodig zijn om een conclusie te kunnen trekken over de oorzaak van LBT-Kanker.

In [377]:
# pie chart voor verdeling emissies per capita en LBT-Kanker
df_sorted = df_emissions_per_capita.sort_values(by='Emissions Per Capita', ascending=False)
fig = px.bar(df_sorted, x='Country', y='Emissions Per Capita', title='Vergelijking emissies per Capita')
fig.show()

df_sorted = df_lung_per_capita.sort_values(by='Lung Cancer Per Capita', ascending=False)
fig = px.bar(df_sorted, x='Country', y='Lung Cancer Per Capita', title='Vergelijking LBT-Kanker per Capita')
fig.show()

# India en Amerika

In de grafiek hieronder zijn twee duidelijke grafieken gemaakt, hierin is heel duidelijk te zien dat het aantal longkanker gevallen flink stijgt als de emissies per capita ook omhoog gaan.

In [378]:
# importeer dataframe
df_combined = pd.read_csv("csv/df/combined_usa_india.csv")

def plot_country_data(df, countries):
    # Create figure with secondary y-axes and two columns
    fig = make_subplots(rows=1, cols=2, specs=[[{"secondary_y": True}, {"secondary_y": True}]],
                        subplot_titles=(countries[0], countries[1]))

    for i, country in enumerate(countries):
        df_country = df[df['Country'] == country]
        # Add traces for Lung Cancer Per Capita
        fig.add_trace(
            go.Scatter(x=df_country['Year'], y=df_country['Lung Cancer Per Capita'], name=f"TBL-Kanker Per Capita - {country}"),
            row=1, col=i+1, secondary_y=False,
        )

        # Add traces for Emissions Per Capita
        fig.add_trace(
            go.Scatter(x=df_country['Year'], y=df_country['Emissions Per Capita'], name=f"Emissies Per Capita - {country}"),
            row=1, col=i+1, secondary_y=True,
        )

        # Set y-axes titles for each subplot
        fig.update_yaxes(title_text="<b>TBL-Kanker Per Capita", row=1, col=i+1, secondary_y=False)
        fig.update_yaxes(title_text="<b>Emissies Per Capita", row=1, col=i+1, secondary_y=True)

    # Update x-axis title and figure title
    fig.update_xaxes(title_text="Jaar", row=1, col=1)
    fig.update_xaxes(title_text="Jaar", row=1, col=2)
    fig.update_layout(
        title_text="Dubbele aslijnen voor USA en India",
        height=600
    )

    fig.show()

# Plot for USA and India
plot_country_data(df_combined, ['USA', 'India'])

# Correlatie plots

Hieronder staat een aantal plots dat correlaties laat zien tussen verschillende variabelen uit onze datasets.

In [379]:
# importeer grouped df
df_grouped = pd.read_csv("csv/df/grouped_df.csv")

# Create the scatter plot with trendline using plotly
correlation_coefficient = df_grouped['EmissionsPerCapita'].corr(df_grouped['LungCancerPerCapita'])
fig = px.scatter(df_grouped, x="EmissionsPerCapita", y="LungCancerPerCapita", trendline="ols",
                 title='Correlatie tussen LBT-kanker en Emissies per Capita (MtCO2 per persoon)',
                 hover_data=["Year"],
                 labels={
                     "EmissionsPerCapita": "Totale Emissies per Capita (MtCO2 per persoon)",
                     "LungCancerPerCapita": "LBT-Kanker Doden per Capita",
                     "Year":"Jaar"
                 })

# Add correlation coefficient as annotation
fig.add_annotation(
    x=max(df_grouped['EmissionsPerCapita']),  # Position the annotation at the far right of the x-axis
    y=min(df_grouped['LungCancerPerCapita']),  # Position the annotation at the bottom of the y-axis
    text=f'Correlation Coefficient: {correlation_coefficient:.2f}',
    showarrow=False,
    font=dict(size=12, color="black"),
    xanchor='right',
    yanchor='bottom'
)

fig.update_layout(
    height = 600
)

# Roken

Roken is de grootste oorzaak van TBL-kanker.
Sigaretten bevatten vele kankerverwekkende stoffen. In de rook van sigaretten zitten meer dan 60 verschillende carcinogenen, waaronder in kleine aantallen een van de sterkste carcinogeen groepen, polycyclic aromatic hydrocarbons (PAH). PAH’s zijn producten van onvolledige verbranding en komen ook voor in uitlaatgassen van onder andere auto’s (Hecht, 2006). 

# Rokers per capita 

Het aantal rokers per capita is een goede aanduiding voor het aantal TBL-Kanker patiënten.

In [380]:
# importeer merged_data
merged_data = pd.read_csv("csv/df/merged_smokers_df.csv")

# Create a choropleth map
fig = px.choropleth(
    merged_data,
    locations="Country",
    locationmode="country names",
    color="Smokers_Per_Capita",
    hover_name="Country",
    color_continuous_scale=px.colors.sequential.Plasma,
    title="Gemiddelde aantal rokers per Capita (2000-2012)"
)

fig.update_layout(
    coloraxis_colorbar=dict(
        title="Rokers<br>Per Capita",
    ),
    geo=dict(
        showframe=False,
        showcoastlines=False,
        projection_type='equirectangular'
    ),
    height=600,    
)
# Show the figure
fig.show()

# Rokers correlatie met LBT-Kanker

In [381]:
# importeer grouped smokers data
df_grouped = pd.read_csv("csv/df/grouped_smokers_df.csv")

# Create the scatter plot with trendline using plotly
correlation_coefficient_smoke = df_grouped['SmokersPerCapita'].corr(df_grouped['LungCancerPerCapita'])
fig = px.scatter(df_grouped, x="SmokersPerCapita", y="LungCancerPerCapita", trendline="ols",
                 title='Correlatie tussen LBT-kanker en Rokers per Capita',
                 hover_data=["Year"],
                 labels={
                     "SmokersPerCapita": "Totale Rokers per Capita",
                     "LungCancerPerCapita": "LBT-Kanker Doden per Capita",
                     "Year":"Jaar"
                 })

# Add correlation coefficient as annotation
fig.add_annotation(
    x=max(df_grouped['SmokersPerCapita']),  # Position the annotation at the far right of the x-axis
    y=min(df_grouped['LungCancerPerCapita']),  # Position the annotation at the bottom of the y-axis
    text=f'Correlation Coefficient: {correlation_coefficient_smoke:.2f}',
    showarrow=False,
    font=dict(size=12, color="black"),
    xanchor='right',
    yanchor='bottom'
)

fig.update_layout(
    height = 600
)



# Referenties 

> Cohen, A. J., & Pope 3rd, C. A. (1995). Lung cancer and air pollution. _Environmental health perspectives, 103_(suppl 8), 219-224.
>
> Safiri, S., Sohrabi, M. R., Carson-Chahhoud, K., Bettampadi, D., Taghizadieh, A., Almasi-Hashiani, A., ... & Kolahi, A. A. (2021). Burden of tracheal, bronchus, and lung cancer and its attributable risk factors in 204 countries and territories, 1990 to 2019. _Journal of Thoracic Oncology, 16_(6), 945-959.
